# Transformer

## refs

### paper
- https://arxiv.org/pdf/1706.03762

### vis
- https://bbycroft.net/llm
- https://docs.google.com/spreadsheets/d/10O-amPDV4zvnZZedlqX33pLFxlO4jj8CbhY6jzdk5rw/edit?gid=260373902#gid=260373902
- https://jalammar.github.io/illustrated-transformer/
- https://jalammar.github.io/illustrated-gpt2/

### impl
- karpathy: https://github.com/karpathy/build-nanogpt
- https://nlp.seas.harvard.edu/annotated-transformer/#attention-visualization
- https://course.fast.ai/Lessons/lesson24.html
- https://github.com/jadore801120/attention-is-all-you-need-pytorch


In [19]:
from IPython.core.display import HTML
HTML(r"""
<style>
    .output-plaintext, .output-stream, .output {
        font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif, "Apple Color Emoji", "Segoe UI Emoji", "Segoe UI Symbol" !important;
        line-height: 1.2 !important;
        font-size: 15px !important;
    }
</style>
""")

# 1. 환경 세팅


- 라이브러리 다운로드

```bash
conda create -n gpt python=3.12 -y
conda activate gpt
# https://github.com/explosion/spaCy/issues/13528
conda install "numpy>=1.19.0,<2.0.0" spacy -y  
conda install pytorch::pytorch torchvision torchaudio torchtext -c pytorch -y
conda install matplotlib tensorboard seaborn -y
```

- gpu setting


In [1]:
#to define the models
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

#to load,iterate and process the dataset
import torchtext
from torchtext.datasets import Multi30k
from torchtext.data.metrics import bleu_score
from torchtext.data import Field,BucketIterator

#to visualize loss plots on localhost while training
# See https://pytorch.org/docs/stable/tensorboard.html for more details
from torch.utils.tensorboard import SummaryWriter

In [2]:
torch.__version__, torchvision.__version__, torchtext.__version__

('2.3.1', '0.18.1', '0.6.0')

In [3]:
#miscallaneous imports

import math
import spacy
import random
import numpy as np
from time import time


from tqdm.notebook import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

sns.set_theme()

In [4]:
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

def setup_device():
    device = get_device()
    print(f"Using device: {device}")

    if device.type == "cuda":
        print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
        print(f"CUDA Version: {torch.version.cuda}")
        torch.cuda.empty_cache()
    elif device.type == "mps":
        print("Using MPS (Metal Performance Shaders) on Mac")
    else:
        print("Using CPU")
    return device

# 사용 예시
device = setup_device()

Using device: mps
Using MPS (Metal Performance Shaders) on Mac


In [5]:
BATCH_SIZE = 128
SEED = 1024

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# 1. 데이터




In [7]:
import os

def load_tokenizers():
    # Download tokenizer from spacy
    try:
        spacy_de = spacy.load("de_core_news_sm") # german
    except IOError:
        os.system("python -m spacy download de_core_news_sm")
        spacy_de = spacy.load("de_core_news_sm")

    try:
        spacy_en = spacy.load("en_core_web_sm") # english
    except IOError:
        os.system("python -m spacy download en_core_web_sm")
        spacy_en = spacy.load("en_core_web_sm")

    return spacy_de, spacy_en


spacy_de, spacy_en = load_tokenizers()

In [8]:
def tokenize_english(eng_text):
    return [token.text for token in spacy_en.tokenizer(eng_text)]

def tokenize_german(german_text):
    return [token.text for token in spacy_de.tokenizer(german_text)]

print(tokenize_english('I wish you all the best'))
print(tokenize_german('Ich wünsche Ihnen alles Gute'))

['I', 'wish', 'you', 'all', 'the', 'best']
['Ich', 'wünsche', 'Ihnen', 'alles', 'Gute']


#### `torchtext.data.Field`

- `lower`: 텍스트를 소문자로 변환할지 여부를 지정합니다.
- `batch_first`: 배치 차원을 첫 번째로 하는 텐서를 생성할지 여부를 지정합니다.


또한 transformer의 encoder에 들어갈 input인 `source`, output인 `target`을 독일어 -> 영어로 지정합니다.

- `source`: german
- `target`: english

In [9]:
from torchtext.data import Field

UNK = '<unk>' # 0
PAD = '<pad>' # 1
SOS = '<sos>' # 2
EOS = '<eos>' # 3

source_process_pipeline = Field(tokenize = tokenize_german,
                       init_token = SOS,
                       eos_token = EOS,
                       pad_token= PAD,
                       unk_token=UNK,
                       lower = True,
                       batch_first = True)
target_process_pipeline = Field(tokenize = tokenize_english,
                       init_token = SOS,
                       eos_token = EOS,
                       pad_token= PAD,
                       unk_token=UNK,
                       lower = True,
                       batch_first = True)

## [Multi30k](https://pytorch.org/text/stable/datasets.html#multi30k)

이제 Multi30k 독일어-영어 번역 작업을 사용한 실제 예제를 살펴보겠습니다. 이 작업은 논문에서 다룬 WMT 작업(대규모 기계 번역 task)보다 훨씬 규모가 작지만, 여전히 전체 시스템을 잘 보여줄 수 있습니다.



In [32]:
# https://github.com/pytorch/text/issues/1756
from torchtext.datasets import Multi30k

Multi30k.urls = [
    "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz",
    "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz",
    "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/mmt_task1_test2016.tar.gz",
]

train_data, validation_data, test_data = Multi30k.splits(exts = ('.de','.en'),
                                                       fields = (source_process_pipeline,
                                                                target_process_pipeline))

In [27]:
source_process_pipeline.build_vocab(train_data,min_freq=2)
target_process_pipeline.build_vocab(train_data,min_freq=2)

print(len(source_process_pipeline.vocab))
print(len(target_process_pipeline.vocab))

7853
5893


In [31]:
print(list(source_process_pipeline.vocab.stoi.items())[:20])
print(list(target_process_pipeline.vocab.stoi.items())[:20])


[('<unk>', 0), ('<pad>', 1), ('<sos>', 2), ('<eos>', 3), ('.', 4), ('ein', 5), ('einem', 6), ('in', 7), ('eine', 8), (',', 9), ('und', 10), ('mit', 11), ('auf', 12), ('mann', 13), ('einer', 14), ('der', 15), ('frau', 16), ('die', 17), ('zwei', 18), ('einen', 19)]
[('<unk>', 0), ('<pad>', 1), ('<sos>', 2), ('<eos>', 3), ('a', 4), ('.', 5), ('in', 6), ('the', 7), ('on', 8), ('man', 9), ('is', 10), ('and', 11), ('of', 12), ('with', 13), ('woman', 14), (',', 15), ('two', 16), ('are', 17), ('to', 18), ('people', 19)]



## [Bucket Iterator](https://torchtext.readthedocs.io/en/latest/data.html?highlight=bucket#bucketiterator)

Pytorch의 dataloader와 비슷한 역할을 한다. 하지만 dataloader 와 다르게 비슷한 길이의 문장들끼리 
batch를 만들기 때문에 padding의 개수를 최소화할 수 있다. 내부적으로 [torchtext.data.pool](https://torchtext.readthedocs.io/en/latest/data.html?highlight=bucket#bucketiterator)을 사용합니다.

In [40]:
train_iterator, validation_iterator, test_iterator = BucketIterator.splits(
    (train_data, validation_data, test_data), 
     batch_size = BATCH_SIZE,
     device = device)

# 2. Model


![](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8f/The-Transformer-model-architecture.png/580px-The-Transformer-model-architecture.png)



## 1-1. Big picture

@TODO encoder-decoder그림
@TODO linearSoftmax 그림
@padding mask 설명, @look-ahead mask 설명

In [26]:
import torch
from torch import nn
from torch.nn.functional import log_softmax


class Transformer(nn.Module):
    def __init__(self, encoder, decoder, i_embed, o_embed, generator):
        super(Transformer, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.i_embed = i_embed
        self.o_embed = o_embed
        self.generator = generator # LinearSoftmax

    def forward(self, inp, oup, in_mask, out_mask):
        """
        Args:
            in_mask (Tensor): Padding mask applied to the input sequence
            out_mask (Tensor): Look-ahead mask applied to the output sequence
        """
        enc_output = self.encoder(self.i_embed(inp), in_mask)
        self.decoder(
            self.o_embed(oup), out_mask, # masked attention return is Q
            enc_output, in_mask # K, V
        )

# Last layer
class LinearSoftmax(nn.Module):
    "Define standard linear + softmax generation step."

    def __init__(self, d_model, vocab):
        super(LinearSoftmax, self).__init__()
        # (... ,in) -> (..., out)
        self.proj = nn.Linear(in_features=d_model, out_features=vocab)

    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1) # out dim(vocab)에 대해서 softmax


## log_softmax
> https://pytorch.org/docs/stable/generated/torch.nn.LogSoftmax.html#torch.nn.LogSoftmax

### 1. log_softmax 사용하는 이유

1. 단조 함수 (monotonic function)
Softmax 함수를 통해 입력 벡터를 확률 분포로 변환한 후, 이 확률 분포에서 가장 높은 값을 가지는 요소, 즉 "max값"을 알아내고자 합니다. 이때 softmax 함수의 결과에 log를 취하면, max값을 찾는 과정에서 변화가 없습니다. 왜냐하면 log 함수는 monotonic한 함수이기 때문에 입력 값의 순서를 유지하며, softmax의 결과에서 가장 큰 값이었던 요소는 log를 취해도 여전히 가장 큰 값으로 남아 있습니다.

2. overflow 방지
또한, log를 취함으로써 얻는 이점 중 하나는 수치적 안정성입니다. softmax 함수는 지수 함수를 포함하므로, 입력 값이 크면 결과값도 매우 크게 증가할 수 있습니다. 
이때 $e^\infty$ 인 경우 numpy float으로 표현할 수 있는 값을 넘어갈 수 있어 overflow가 발생할 수 있습니다. 이를 방지하기 위해서 pytorch에서는 `log_softmax()`, `logsumexp()`를 제공합니다.

따라서 log(softmax) 함수는 softmax의 확률 값을 변환하되, 확률 분포에서의 max값을 유지하며, 수치적으로 안정성이 더 좋은 결과를 제공하는 함수로 사용될 수 있습니다.

다음으로 어떻게 log_softmax가 exp를 안정적으로 계산하여, overflow를 방지하고 있는지를 살펴보도록 하겠습니다.

로그를 취하면 큰 값이 덧셈으로 변환되므로 수치적으로 안정한 계산이 가능합니다.


## 2. log\_softmax 수식

log\_softmax 함수는 주어진 벡터 $\mathbf{x} = [x_1, x_2, \ldots, x_n]$에 대해 pytorch에서는 다음과 같이 계산됩니다:

$$
\text{log\_softmax}(x_i) = x_i - c - \log \left( \sum_{j=1}^{n} e^{x_j - c} \right)
$$

이때 $c$는 벡터 $ \mathbf{x} $의 최대값이라고 정의합니다.

$ c = \max(x_1, x_2, \ldots, x_N) $

c를 빼줌으로써, $e^\text{매우큰수}$ 인 경우를 배제하여 overflow를 방지할 수 있습니다. 

그럼 이제 위의 수식이 어떻게 나오게 되었는지 아래에서 확인 해보겠습니다.


### 원래의 소프트맥스 수식

소프트맥스 함수는 다음과 같이 정의됩니다:

$$
\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_{j=1}^{n} e^{x_j}}
$$

### 로그 소프트맥스

소프트맥스 함수의 출력에 로그를 취하면 다음과 같습니다:

$$
\log \left( \text{softmax}(x_i) \right) = \log \left( \frac{e^{x_i}}{\sum_{j=1}^{n} e^{x_j}} \right)
$$

로그의 성질을 이용하여 분자와 분모를 분리하면:

$$
\log \left( \text{softmax}(x_i) \right) = \log(e^{x_i}) - \log \left( \sum_{j=1}^{n} e^{x_j} \right)
$$

이는 다음과 같이 단순화됩니다:

$$
\log \left( \text{softmax}(x_i) \right) = x_i - \log \left( \sum_{j=1}^{n} e^{x_j} \right)
$$

하지만 여전히 뒤에, exp(x)에서 x가 무한히 클 경우 overflow가 발생할 수 있습니다. 이렇게 뒤쪽에 있는 log sum exp를 pytorch에서는 [torch.logsumexp](https://pytorch.org/docs/stable/generated/torch.logsumexp.html)로 구현해주고 있으며, 아래와 같이 trick을 사용해서 계산합니다.

### logsumexp

`logsumexp`를 구현해주기 위해서, 먼저, $ c $를 벡터 $ \mathbf{x} $의 최대값이라고 정의합니다:

$ c = \max(x_1, x_2, \ldots, x_N) $

이후 아래의 지수함수 성질을 이용합니다.

$ e^{a + b} = e^{a} \cdot e^{b} $

지수함수 성질을 통해, 원래 수식을 다음과 같이 재정렬할 수 있습니다:
$ e^{x_n} = e^{(x_n - c) + c} = e^{x_n - c} \cdot e^{c} $

#### 3. 지수 함수 성질 적용
위의 성질을 원래 수식에 적용해봅시다:
$ \sum_{n=1}^{N} e^{x_n} = \sum_{n=1}^{N} e^{x_n - c} \cdot e^{c} $

여기서 $e^{c}$는 상수이므로 sigma 밖으로 빼낼 수 있습니다:
$ \sum_{n=1}^{N} e^{x_n} = e^{c} \sum_{n=1}^{N} e^{x_n - c} $

이를 다시 정리하면:

$$ 
sumexp = \sum_{n=1}^{N} e^{x_n} = e^{c} \sum_{n=1}^{N} e^{x_n - c}
$$


$$
\log (sumexp) = \log \left( e^c \sum_{j=1}^{n} e^{x_j - c} \right) = c + \log \left( \sum_{j=1}^{n} e^{x_j - c} \right)
$$

이를 이용하여 log\_softmax 수식을 다시 쓰면:

$$
\text{log\_softmax}(x_i) = x_i - \log (sumexp)
$$

$$
\text{log\_softmax}(x_i) = x_i - c - \log \left( \sum_{j=1}^{n} e^{x_j - c} \right)
$$

이 변형을 통해 매우 큰 값에 대해 수치적으로 안정적인 계산이 가능하게 됩니다.


## c.f softmax vs log_softmax

이제 softmax와 log_softmax를 비교해보겠습니다.

In [27]:
import torch
import torch.nn.functional as F

inputs = torch.tensor([1000.0, 2000.0, 3000.0])

def softmax(x):
    exp_x = torch.exp(x)
    sum_exp_x = torch.sum(exp_x)
    return exp_x / sum_exp_x

def log_softmax(x):
    c = torch.max(x)
    logsumexp = torch.log(torch.sum(torch.exp(x - c)))
    return x - c - logsumexp


print("Softmax output (overflow):", softmax(inputs))

# log_softmax_output = F.log_softmax(inputs, dim=0)
print("Log softmax output (stability):", log_softmax(inputs))

Softmax output (overflow): tensor([nan, nan, nan])
Log softmax output (stability): tensor([-2000., -1000.,     0.])


## 1-2. Encoder layer

The encoder is composed of a stack of N = 6 identical layers.



### Residual Dropout (5.4)
>  We apply dropout [27] to the output of each sub-layer, before it is added to the
sub-layer input and normalized. In addition, we apply dropout to the sums of the embeddings and the
positional encodings in both the encoder and decoder stacks.


In [54]:
import copy

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


class Encoder(nn.Module):
    "Core encoder is a stack of N layers"

    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, padding_mask):
        for layer in self.layers:
            x = layer(x, padding_mask) # norm -> dropout
        return self.norm(x)


# Layer norm test
class LayerNorm(nn.Module):
    """Construct a layernorm module

    d_model 방향으로 평균, 표준편차 계산한다.
    """

    def __init__(self, features, eps=1e-5):
        super(LayerNorm, self).__init__()
        self.w = nn.Parameter(torch.ones(features))
        self.b = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True) # x is (batch, sentence, d_model)
        std = x.std(-1, keepdim=True)
        return self.w * (x - mean) / (std + self.eps) + self.b


## 논문 상에서 Layer normalization

> Unless otherwise noted, the default initialization of layer normalization is to set the adaptive gains to 1 and the biases to 0  ([Layer normalization 6.](https://arxiv.org/pdf/1607.06450))

- alpha: 1
- beta: 0

구현한 layer normalization을 pytorch 구현체와 비교해봅니다.

In [55]:
batch_size = 2
seq_length = 3
d_model = 512
input_tensor = torch.randn(batch_size, seq_length, d_model)
print(LayerNorm(d_model)(input_tensor))
# check pytorch layer norm
print(nn.LayerNorm(d_model)(input_tensor)) # random seed 영향

torch.Size([2, 3, 1])
tensor([[[-0.8275, -0.0135,  0.6378,  ..., -1.2619,  0.0085, -1.2845],
         [ 0.5016, -1.5572,  0.2296,  ...,  0.3337, -0.2966, -0.9650],
         [-0.0318,  0.4579, -1.8165,  ..., -0.1260,  0.7774, -0.2929]],

        [[ 0.9011, -0.1162, -1.1876,  ..., -2.0658, -1.3579,  2.4320],
         [-0.2494,  1.5145,  0.6840,  ...,  0.0284, -0.7935,  0.1723],
         [-1.3209,  0.4843,  1.5879,  ...,  0.9951, -1.4229, -1.6999]]],
       grad_fn=<AddBackward0>)
tensor([[[-0.8283, -0.0135,  0.6384,  ..., -1.2632,  0.0085, -1.2858],
         [ 0.5021, -1.5587,  0.2298,  ...,  0.3341, -0.2969, -0.9660],
         [-0.0318,  0.4583, -1.8183,  ..., -0.1261,  0.7782, -0.2931]],

        [[ 0.9020, -0.1163, -1.1888,  ..., -2.0678, -1.3593,  2.4344],
         [-0.2497,  1.5159,  0.6847,  ...,  0.0285, -0.7942,  0.1725],
         [-1.3222,  0.4848,  1.5895,  ...,  0.9961, -1.4243, -1.7016]]],
       grad_fn=<NativeLayerNormBackward0>)


## Layer normalization vs Batch normalization

![](https://i.sstatic.net/E3104.png)

위 그림에서 Feature를 d_model로 생각하면, layer normalization의 경우, 한 문장의 한 토큰이 가진 d_model들의 평균과 분산을 구해서 normalization을 진행합니다. 이는 배치와 문장들에 상관없이 평균 / 분산이 계산됩니다. `(batch, seq_len, 1)`

Batch normalization의 경우, 하나의 배치안에 존재하는 모든 sentence 문장들에 대해서 평균을 구합니다. 즉 "love"라는 단어와, "hate"라는 단어가 각각 512차원으로 하나의 배치의 문장들에 포함되어있을 경우, 0~511 index 각각 값들을 모든 배치안의 모든 문장안의 모든 단어들에 대해서 값들을 얻어와 평균과 분산을 계산합니다. `(1,512)`


## 아래 코드 적용
주어진 input tensor input_tensor은 (batch_size, seq_length, d_model)의 형태에 대하여

`Layer normalization`은 d_model 차원에 대해 각각의 샘플(문장)에 대해 평균과 표준 편차를 계산합니다. 이를 통해 (batch_size, seq_length, 1)의 평균, 표준편차가 만들어집니다. 이 과정을 통해 각 샘플의 각 feature dimension이 평균 0, 분산 1에 가까워지게 됩니다.


`Batch normalization`은 한 번에 전체 batch의 모든 샘플에 대해 각 feature dimension을 정규화합니다. 따라서 입력의 모든 샘플에 대해 평균과 표준 편차를 계산하고, 이를 사용하여 정규화를 수행합니다. 이를 통해 평균과 표준편차는 (1, 1, d_model)의 shape를 가집니다. Batch normalization은 전체 batch의 모든 샘플에 대해 계산하기 때문에, 데이터의 분포를 안정화시키고 학습 과정을 안정화시키는 데 도움을 줍니다.


**Layer normalization 처럼 각 토큰들에 대해서 독립적으로 평균 / 분산을 계산해서 normalization을 하는 것이 nlp에서는 실험결과 더 효과적이라고 평가합니다.**

In [42]:
def test_custom_init(batch_size, seq_length, d_model):
    input_tensor = torch.zeros(batch_size, seq_length, d_model)
    for i in range(seq_length):
        init_value = i + 1
        input_tensor[:, i, :] = init_value  # 각 seq_length에 따라 d_model의 값을 초기화
    return input_tensor

input_tensor = test_custom_init(batch_size, seq_length, d_model)
print(input_tensor)

output_tensor = LayerNorm(d_model)(input_tensor)
print("Output Tensor:", output_tensor)

# check pytorch layer norm
nn.LayerNorm(d_model)(input_tensor)

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [2., 2., 2.,  ..., 2., 2., 2.],
         [3., 3., 3.,  ..., 3., 3., 3.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [2., 2., 2.,  ..., 2., 2., 2.],
         [3., 3., 3.,  ..., 3., 3., 3.]]])
torch.Size([2, 3, 1])
Output Tensor: tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], grad_fn=<AddBackward0>)


tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], grad_fn=<NativeLayerNormBackward0>)

In [53]:
# Compare with batch norm

# pytorch BatchNorm1d input (batch, features seq_length) 
# (2,3,512) -> (2,512,3)
input_tensor_transposed = input_tensor.transpose(1, 2) #  (batch_size, seq_length, d_model) -> (batch_size, d_model, seq_length)
print(input_tensor_transposed.shape)
print(input_tensor_transposed)
nn.BatchNorm1d(d_model)(input_tensor_transposed).transpose(1,2)

torch.Size([2, 512, 3])
tensor([[[1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.],
         ...,
         [1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.]],

        [[1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.],
         ...,
         [1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.]]])


tensor([[[-1.2247, -1.2247, -1.2247,  ..., -1.2247, -1.2247, -1.2247],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.2247,  1.2247,  1.2247,  ...,  1.2247,  1.2247,  1.2247]],

        [[-1.2247, -1.2247, -1.2247,  ..., -1.2247, -1.2247, -1.2247],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.2247,  1.2247,  1.2247,  ...,  1.2247,  1.2247,  1.2247]]],
       grad_fn=<TransposeBackward0>)

In [33]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."

    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = deepcopy(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"

    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = deepcopy(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)    